In [5]:
get_ipython().run_line_magic('matplotlib', 'inline')

import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt # Needed for visualization
import matplotlib as mpl # Needed for visualization
from mpld3 import * # Needed for visualization
import datapane as dp # Needed for visualization
import altair as alt # Needed for visualization
from flask import Flask, request, render_template, redirect, url_for # Needed for web implementation
from sklearn.feature_extraction.text import TfidfVectorizer # Needed for recommendation algorithm
from sklearn.metrics.pairwise import cosine_similarity # Needed for similarity calculation

In [ ]:
#enable automatic D3 display of prepared model data 
enable_notebook()

# set the value of maximum displayed rows to 50 rows
pd.set_option('display.max_rows', 50)

np.set_printoptions(threshold=sys.maxsize)
alt.data_transformers.disable_max_rows()

In [ ]:
# function for setting up the scatter plot
def sc1(results):
    sc1_results = results[results["rating"].notna()]
    sc1_results = sc1_results.reset_index().iloc[:,1:]

    fig, ax = plt.subplots(figsize = (12,12))
    # Defines variables to use
    points = ax.scatter(sc1_results["rating"], sc1_results["similarity"],
                        s=400, alpha=0.7)

    # Labels plot and sets ranges
    labels = [", ".join([sc1_results["major_name"][i], sc1_results["university"][i]]) for i in range(len(sc1_results.index))]
    tooltip = plugins.PointLabelTooltip(points, labels)
    ax.set_xlim(0, 5)
    ax.set_ylim(0, sc1_results["similarity"].max() + 0.04)
    ax.set_title("Recommended majors by overall rating and similarity")
    ax.set_xlabel("Ratings from students")
    ax.set_ylabel("Degree of similarity to your interests")
    ax.tick_params(axis="both", which="major", labelsize=18)

    # Creates hovering interactivity
    plugins.connect(fig, tooltip)

    # Saves plot to html for web view
    save_html(fig, "sc1_plot.html")
    sc1_plot = fig_to_html(fig)
    return sc1_plot

In [ ]:
# function for setting up the bar chart

def bar1(results):
    subcategory_list = list(results['language'].unique())
    # Sets variables
    results["major"] = results["major_name"] + ", " + results["university"] + " (" + results["degree_type"] + ")                                 " + results["major_id"].astype(str)

    # Creates dropdown interactivity
    input_dropdown = alt.binding_select(options=subcategory_list)
    selection = alt.selection_single(fields=['language'], bind=input_dropdown, name='Filter by: ')
    selection2 = alt.selection_multi(fields=["subcategory"], bind = "legend")

    # Labels plot and defines optics
    alt_plot = alt.Chart(results).mark_bar().encode(
        x='similarity',
        y=alt.Y("major", sort = ["rank"]),
        color=alt.Color("subcategory:N", scale=alt.Scale(scheme="category20b")),
    ).properties(
        width = 600
    ).configure_axis(
        labelFontSize=18,
        labelLimit=350
    ).add_selection(
        selection
    ).transform_filter(
        selection
    )

    # Saves plot as html for web view
    alt_plot.save("bar1.html")
    with open('bar1.html', 'r') as f:
        bar1_html = f.read()
    return bar1_html

In [ ]:
%run ../model/recommender.ipynb

In [ ]:
app = Flask(__name__)

# Route for handling the Home Page
@app.route('/')

# render the Home Page html template
def my_form():
    return render_template("Home.html")

In [ ]:
# Route for handling the Results Page
@app.route('/results', methods = ["POST"])

# Function to display the table, a scatter plot and a bar plot
def results():
    global text
    global results
    text = request.form["interests"]
    results = studybuddy(text, 100)
    sc1_plot = sc1(results=results)
    bar1_plot = bar1(results=results)
    
# Renders the Results Page html template
    return render_template('results.html', title='Your Study Buddy Results', majors=results, sc1_plot = sc1_plot, bar1_plot=bar1_plot)


In [ ]:
# Route for handling the filtered Results page
@app.route('/rld', methods = ["POST"])

# Function to display a filtered table, a scatter plot and a bar plot
def rld():
    form = request.form["nresultsString"]
    if request.method == "POST":
        nresultsString = request.form["nresultsString"]
        results = studybuddy(text, int(nresultsString))
        sc1_plot = sc1(results=results)
        bar1_plot = bar1(results=results)
        
# Render the Results Page extension html template
    return render_template('results.html',
                           title='Your Study Buddy Results',
                           majors=results,
                           sc1_plot = sc1_plot,
                           bar1_plot=bar1_plot
                           )

In [ ]:
# Initializes web app
if __name__ == '__main__':
    app.run()